In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
def move_column(df, cn, cp):
    # 定义移动的列
    m_column = df.pop(cn)
    # 关键代码(位置, 列名, 要移动的列) 
    df.insert(cp, cn, m_column)
    return df
    # 打印提示信息
    print('列位置更改成功')

# # 调用方法
# move_column(file_move, column_name, column_position)

In [4]:
# 读txt文件
def readPatList(pat_file_list):
    ret_list_list=[]
    for i in pat_file_list:
        patlist = pd.read_table(i,sep='\t',header=None,index_col=None)[0].tolist()
        ret_list_list.append(patlist)
    
    ret_list =[]
    for i in ret_list_list:
        for j in i:
            ret_list.append(j)
    
    ret_list_ret = list(set(ret_list))
    
    print(f'读取 {len(pat_file_list)} 个文件,重合 {len(ret_list)-len(ret_list_ret)},共 {len(ret_list_ret)} 个病人')
    return ret_list_ret

# readPatList([r'../../分组/jsph_gbm.txt',r'../../分组/jsph_lgg.txt',
#                                      r'../../分组/TEST_jsph_gbm.txt',r'../../分组/TEST_jsph_lgg.txt'])

# 获取CIBERSORT和radiomics数据的交叉

In [5]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  打开影像组学文件
df_radiomics = pd.read_table('20_RiskScore_radiomics1402_pretreated_OS.csv',sep=',',index_col=0)
feature_not_radiomics = df_radiomics.columns.tolist()[:3]
print(feature_not_radiomics)
df_radiomics

['status', 'time', 'RiskScore_h0']


,status,time,RiskScore_h0,log.sigma.5.mm.3D_glcm_Idn,squareroot_firstorder_Median,lbp.3D.k_glrlm_ShortRunEmphasis,wavelet.LLH_glszm_GrayLevelNonUniformity,log.sigma.5.mm.3D_firstorder_90Percentile,wavelet.LHH_ngtdm_Contrast,square_glrlm_RunEntropy,...,original_shape_MinorAxisLength,wavelet.LLH_firstorder_Mean,original_shape_Sphericity,wavelet.LLL_glcm_Idmn,wavelet.LHL_gldm_DependenceVariance,wavelet.HLL_gldm_DependenceVariance,log.sigma.3.mm.3D_glszm_ZoneEntropy,log.sigma.5.mm.3D_firstorder_Median,lbp.2D_firstorder_InterquartileRange,wavelet.LHH_gldm_DependenceNonUniformityNormalized
X0907236_ZhouJiSheng,1,6.83,0.896298,0.978929,418.103580,0.419132,73.934508,13.716698,0.001638,4.957925,...,56.688722,-1.006697,0.498970,0.998059,29.992383,33.763932,6.205362,-24.819478,2.5,0.104869
X0875917_LiYan,1,15.70,0.901296,0.976438,469.206017,0.372534,48.381674,18.177860,0.000817,4.973058,...,59.609625,-0.910070,0.662562,0.998251,33.161329,33.761676,6.552695,-29.572262,2.0,0.078167
TCGA-27-1830,1,5.13,2.621098,0.976809,361.326537,0.392058,91.859655,-1.832864,0.003141,4.811214,...,33.948305,-2.664174,0.616121,0.997026,28.714363,30.516095,5.870566,-48.545515,2.0,0.080178
TCGA-DU-5854,0,8.57,0.627098,0.975821,378.441206,0.297715,13.598095,9.866241,0.001674,4.827622,...,42.470084,-0.894696,0.663250,0.997674,27.613160,34.625264,5.097847,-20.290770,2.0,0.080079
HF1588,0,74.20,1.181956,0.968646,389.782128,0.409873,52.251438,12.997869,0.019593,4.761555,...,37.550941,-0.326498,0.531358,0.996102,25.230655,27.546266,5.379736,-40.516292,3.0,0.078900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X1052598_ShiHongMei,0,29.67,0.619839,0.973971,545.344729,0.405578,66.154018,-6.453781,0.001158,5.066364,...,51.721233,-1.941097,0.682478,0.998560,30.617479,34.898009,6.441233,-57.654155,3.0,0.085675
X0930877_XuPingLan,1,8.63,1.547634,0.980352,496.023289,0.296317,31.711602,19.271812,0.010592,5.255231,...,53.134315,-0.414315,0.613724,0.998826,33.895249,34.235136,6.592257,-24.207621,2.0,0.081979
TCGA-DU-6395,1,49.70,2.314271,0.977762,416.562123,0.339940,437.999479,4.845201,0.002410,4.611289,...,54.899323,-1.953735,0.373961,0.996877,33.189645,39.412890,4.749348,-26.371862,2.0,0.072793
HF1587,0,74.20,1.226855,0.972784,447.538339,0.349452,85.220925,6.252650,0.001351,4.968237,...,41.525773,-1.323163,0.691430,0.997400,30.526584,34.167278,5.717379,-53.489113,2.0,0.079584


In [7]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  打开ESTIMATE结果文件
df_estimate = pd.read_table(r'../1.免疫评分计算/ALL_estimate.csv',sep=',',index_col=0)
df_estimate

,StromalScore,ImmuneScore,ESTIMATEScore,TumorPurity
X1002249_GuCongXiang,-1100.533393,201.591404,-898.941989,0.890191
X1010577_LiXiuJin,-1164.245784,101.016880,-1063.228904,0.900917
X1027770_LiuYueQin,-832.834035,451.074995,-381.759040,0.853073
X1029842_XuLongSheng,-830.881381,190.789809,-640.091572,0.872242
X1031574_LiLinSheng,-1021.728364,-142.266401,-1163.994766,0.907238
...,...,...,...,...
TCGA-DU-6402,-27.340953,961.160861,933.819907,0.737077
TCGA-QH-A6X8,-1137.183070,-409.991380,-1547.174451,0.929449
TCGA-DU-A5TY,168.596353,1326.309434,1494.905787,0.678981
TCGA-DU-8163,-420.843431,597.733755,176.890324,0.807466


In [8]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  获取交叉
df_estimate_list = df_estimate.index.tolist()
df_radiomics_list = df_radiomics.index.tolist()
patCrossCondition = list(set(df_radiomics_list) & set(df_estimate_list))
print(f'有{len(patCrossCondition)}个交叉')

df_tmp = pd.concat([df_estimate,df_radiomics],axis=1).T[patCrossCondition].T
df_tmp

有253个交叉


,StromalScore,ImmuneScore,ESTIMATEScore,TumorPurity,status,time,RiskScore_h0,log.sigma.5.mm.3D_glcm_Idn,squareroot_firstorder_Median,lbp.3D.k_glrlm_ShortRunEmphasis,...,original_shape_MinorAxisLength,wavelet.LLH_firstorder_Mean,original_shape_Sphericity,wavelet.LLL_glcm_Idmn,wavelet.LHL_gldm_DependenceVariance,wavelet.HLL_gldm_DependenceVariance,log.sigma.3.mm.3D_glszm_ZoneEntropy,log.sigma.5.mm.3D_firstorder_Median,lbp.2D_firstorder_InterquartileRange,wavelet.LHH_gldm_DependenceNonUniformityNormalized
TCGA-CS-4943,-732.898767,-19.587110,-752.485877,0.880191,1.0,36.87,3.514598,0.979538,318.613831,0.243807,...,51.563073,-1.290122,0.666655,0.997906,27.480214,26.813274,5.203181,-19.939854,2.0,0.096912
HF1511,-527.407770,-446.832926,-974.240696,0.895172,1.0,55.80,2.385795,0.972955,440.472950,0.410223,...,51.586029,-1.008805,0.463937,0.997334,27.890943,33.483033,6.137163,-44.810083,2.5,0.081686
TCGA-HT-7602,-632.838963,141.896642,-490.942321,0.861326,0.0,30.27,0.845872,0.972958,386.150906,0.346306,...,49.028322,-0.622467,0.543791,0.998275,31.552612,35.635830,5.756778,-20.974491,2.0,0.085408
HF1150,-771.353295,-53.069854,-824.423149,0.885154,1.0,20.90,1.839389,0.980919,401.487404,0.359537,...,50.972734,-0.703024,0.577285,0.997107,25.238326,28.392871,6.190767,-22.950979,2.0,0.087009
HF1538,200.615483,331.881734,532.497217,0.775587,1.0,3.20,10.670752,0.979149,356.775323,0.269807,...,57.819405,-0.699642,0.547197,0.998435,27.389213,27.656481,6.242743,-17.429855,2.5,0.091665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-DU-8168,-1503.076894,-949.233019,-2452.309913,0.970133,0.0,14.37,0.189589,0.980137,451.805622,0.459723,...,65.126151,-1.848521,0.597168,0.998625,29.806893,33.541213,6.243592,-19.413232,3.0,0.088857
TCGA-DU-A5TP,-349.821384,1366.845690,1017.024307,0.728768,0.0,33.73,1.653787,0.979027,378.291931,0.336337,...,64.597601,-0.902561,0.514699,0.999127,32.489030,35.461165,6.164322,-14.811944,2.0,0.072234
TCGA-DU-5872,-332.290748,490.109521,157.818773,0.809114,0.0,17.73,0.379153,0.970462,504.334720,0.333477,...,53.137621,-2.141038,0.733167,0.996321,26.272064,30.347943,5.767697,-59.855883,2.0,0.084174
TCGA-DU-8162,-626.952153,-507.662658,-1134.614811,0.905415,1.0,14.80,0.124831,0.979850,496.198792,0.360231,...,34.812874,-1.104114,0.644619,0.997529,35.702282,40.376268,5.587508,-40.534438,2.0,0.073362


In [11]:
# 要把 数据列 变成分类的 特征名们
features_all = list(set(df_tmp.columns.tolist()) - set(feature_not_radiomics))
print(f'【{df_tmp.shape[0]}行数据】*【{len(features_all)}】=【{df_tmp.shape[0]*len(features_all)}行数据】')

# 开始处理
df_list = []
reserveList=list(set(feature_not_radiomics))

for i in features_all:
    df_single = df_tmp[reserveList]  # 原来的列
    new_col = df_tmp[i].values.tolist()  # 新列

    # 在第一列加入新的
    df_single.insert(0, 'enrichment_value', new_col)
    # 把分类重复n次之后加入df中
    class_col = [i for j in range(df_tmp.shape[0])]
    df_single.insert(0, 'class', class_col)

    df_list.append(df_single)
    
df_list[0]

【253行数据】*【24】=【6072行数据】


,class,enrichment_value,RiskScore_h0,status,time
TCGA-CS-4943,TumorPurity,0.880191,3.514598,1.0,36.87
HF1511,TumorPurity,0.895172,2.385795,1.0,55.80
TCGA-HT-7602,TumorPurity,0.861326,0.845872,0.0,30.27
HF1150,TumorPurity,0.885154,1.839389,1.0,20.90
HF1538,TumorPurity,0.775587,10.670752,1.0,3.20
...,...,...,...,...,...
TCGA-DU-8168,TumorPurity,0.970133,0.189589,0.0,14.37
TCGA-DU-A5TP,TumorPurity,0.728768,1.653787,0.0,33.73
TCGA-DU-5872,TumorPurity,0.809114,0.379153,0.0,17.73
TCGA-DU-8162,TumorPurity,0.905415,0.124831,1.0,14.80


In [12]:
# 把散装的合并起来
df_long = pd.concat(df_list,axis =0,ignore_index=False)
df_long

df_long.to_csv('→长数据_供绘散点图.csv',index=True,header=True)

# 重构（以下废弃）

In [9]:
# # 打开文件位置
# jsph_lgg = readPatList([r'../../分组/jsph_lgg.txt'])
# jsph_gbm = readPatList([r'../../分组/jsph_gbm.txt'])
# TEST_jsph_lgg = readPatList([r'../../分组/TEST_jsph_lgg.txt'])
# TEST_jsph_gbm = readPatList([r'../../分组/TEST_jsph_gbm.txt'])
# jsph = readPatList([r'../../分组/jsph_lgg.txt',r'../../分组/jsph_gbm.txt',
#                    r'../../分组/TEST_jsph_lgg.txt',r'../../分组/TEST_jsph_gbm.txt'])

# rbrt_lgg = readPatList([r'../../分组/rbrt_lgg.txt'])
# rbrt_gbm = readPatList([r'../../分组/rbrt_gbm.txt'])
# rbrt = readPatList([r'../../分组/rbrt_lgg.txt',r'../../分组/rbrt_gbm.txt'])

# tcga_lgg = readPatList([r'../../分组/tcga_lgg.txt'])
# tcga_gbm = readPatList([r'../../分组/tcga_gbm.txt'])
# tcga = readPatList([r'../../分组/tcga_lgg.txt',r'../../分组/tcga_gbm.txt'])

In [10]:
# def main(file_path,save_name):
#     print(f'处理【{save_name}】')
#     # 读取病人列表
#     pat_list = readPatList(file_path)
#     # 取交集
#     pat_cross = list(set(pat_list) & set(df_tmp.index.tolist()))
    
#     df_long_matched = df_long.T[pat_cross].T
#     print(f'交集{len(df_mid)}人')
    
# #     save_name = r'../→CIBERSORT/→'+save_name+'_matched.csv'
# #     df_mid.to_csv(save_name,index=True,header=True)
    

In [11]:
# main([r'../../分组/tcga_lgg.txt'],'tcga_lgg')